크롤링할 데이터 위치까지 selenium

In [3]:
# selenium 실행문
from selenium import webdriver
# chromedriver 실행문
from selenium.webdriver.chrome.service import Service

# 변수 driver를 chromedriver에 연결
driver = webdriver.Chrome(service=Service('../driver/chromedriver'))
# 변수 driver에 daum finance 링크 가져와서 연결
driver.get('https://finance.daum.net/')
# selenium으로 실행한 화면 최대화
driver.maximize_window()

In [4]:
# selenium의 by 실행문
from selenium.webdriver.common.by import By

# 코스피 페이지 실행
Kospi_banner = driver.find_element(By.XPATH, '''//*[@id="boxIndexes"]/div[1]/h5/a''')
Kospi_banner.click()

In [5]:
# 원하는 위치까지 밑으로 스크롤링
driver.execute_script('window.scrollTo(0, 1300)')

데이터 추출 1차 - Page 01 ~ 10 df : result_df_1to10

In [6]:
# 추출하기 시작할 페이지 이외의 위치로 이동 - 추출하기 시작할 페이지에서 시작 시 해당 페이지는 생략

page_2 = driver.find_element(By.XPATH, '''//*[@id="boxDailyHistory"]/div[2]/div/div/a[1]''')
page_2.click()

In [7]:
#다음 페이지 버튼까지 누르고 데이터도 추출(페이지6_23.12.28~페이지10_까지)

import time
from bs4 import BeautifulSoup
import pandas as pd

# 결과를 저장할 빈 데이터프레임 생성
result_df_1to10 = pd.DataFrame(columns=[
    '일자', '종가', '전일비', '등락률', 
    '거래량', '거래대금', '개인(억)', '외국인(억)', '기관(억)'
])

for i in range(1, 11):  
    css_selector = f"#boxDailyHistory > div.box_contents > div > div > a:nth-child({i})"
    page_elements = driver.find_elements(By.CSS_SELECTOR, css_selector)
    for element in page_elements:
        element.click()
        time.sleep(1)
        
        # 변수 page를 driver로 열람 중인 페이지 소스로 설정
        page = driver.page_source
        # 변수 soup은 HTML 문서를 파싱해서 bs구조로 설정
        soup = BeautifulSoup(page, 'html.parser')

        # div_history 변수를 변수 soup이 daum finance 속 추출 목표 div class임을 설정
        div_history = soup.find('div', id='boxDailyHistory')

        # 가져올 데이터 정리를 위해 변수 data 뒤에 nn번호 지정하여 컬럼 정리
        ## 사용하지 않을 컬럼을 빼고 가져오면 에러 발생
        data = {
            '일자': [],
            '종가': [],
            '전일비': [],
            '등락률': [],
            '거래량': [],
            '거래대금': [],
            '개인(억)': [],
            '외국인(억)': [],
            '기관(억)': []
        }

        # 변수 rows는 div_history 변수 속 태그 'tr'을 행에
        rows = div_history.find_all('tr')
        for row in rows[1:]:
            cell = row.find_all('td')
            data['일자'].append(cell[0].get_text(strip=True))
            data['종가'].append(cell[1].get_text(strip=True))
            data['전일비'].append(cell[2].get_text(strip=True))
            data['등락률'].append(cell[3].get_text(strip=True))
            data['거래량'].append(cell[4].get_text(strip=True))
            data['거래대금'].append(cell[5].get_text(strip=True))
            data['개인(억)'].append(cell[6].get_text(strip=True))
            data['외국인(억)'].append(cell[7].get_text(strip=True))
            data['기관(억)'].append(cell[8].get_text(strip=True))

        # 데이터프레임 생성 후 결과 데이터프레임에 추가
        df = pd.DataFrame(data)
        result_df_1to10 = pd.concat([result_df_1to10, df], ignore_index=True)

    # 페이지가 10의 배수일 때 다음 페이지로 넘어가기
    if i % 10 == 0:
        next_button = driver.find_element(By.CSS_SELECTOR, '#boxDailyHistory > div.box_contents > div > div > a.btnNext')
        next_button.click()
        time.sleep(1)

# 결과 데이터프레임 출력
result_df_1to10.head()


/tmp/ipykernel_29324/845585893.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,일자,종가,전일비,등락률,거래량,거래대금,개인(억),외국인(억),기관(억)
0,24.03.22,"2,748.56",▼6.30,-0.23%,"501,319","11,385,951","-5,299","6,649","-1,630"
1,24.03.21,"2,754.86",▲64.72,+2.41%,"825,227","14,013,557","-29,144","18,706","10,506"
2,24.03.20,"2,690.14",▲33.97,+1.28%,"720,452","12,739,395","-14,054","8,516","5,866"
3,24.03.19,"2,656.17",▼29.67,-1.10%,"695,866","10,702,805","9,226","-2,388","-7,013"
4,24.03.18,"2,685.84",▲19.00,+0.71%,"408,182","9,523,806","-1,865",-847,"2,254"


In [8]:
len(result_df_1to10)

100

데이터 추출 2차 - Page 11 ~ 530 df: result_df_11to530

In [9]:
#i max = 13 이어서 원하는 2003.01.02 값까지 추출 불가능
#최상위에 for 구문 삽입하여 52번 반복

import time
from bs4 import BeautifulSoup
import pandas as pd

# 결과를 저장할 빈 데이터프레임 생성
result_df_11to530 = pd.DataFrame(columns=[
    '일자', '종가', '전일비', '등락률', 
    '거래량', '거래대금', '개인(억)', '외국인(억)', '기관(억)'
])



for _ in range(52):
    click_other_page = driver.find_element(By.CSS_SELECTOR, '#boxDailyHistory > div.box_contents > div > div > a:nth-child(8)')
    click_other_page.click()
    for i in range(3, 13): 
        css_selector = f"#boxDailyHistory > div.box_contents > div > div > a:nth-child({i})"
        time.sleep(1)
        page_elements = driver.find_elements(By.CSS_SELECTOR, css_selector)
        for element in page_elements:
            element.click()
            time.sleep(1)

            # 변수 page를 driver로 열람 중인 페이지 소스로 설정
            page = driver.page_source
            # 변수 soup은 HTML 문서를 파싱해서 bs구조로 설정
            soup = BeautifulSoup(page, 'html.parser')

            # div_history 변수를 변수 soup이 daum finance 속 추출 목표 div class임을 설정
            div_history = soup.find('div', id='boxDailyHistory')

            # 가져올 데이터 정리를 위해 변수 data 뒤에 nn번호 지정하여 컬럼 정리
            ## 사용하지 않을 컬럼을 빼고 가져오면 에러 발생
            data = {
                '일자': [],
                '종가': [],
                '전일비': [],
                '등락률': [],
                '거래량': [],
                '거래대금': [],
                '개인(억)': [],
                '외국인(억)': [],
                '기관(억)': []
            }

            # 변수 rows는 div_history 변수 속 태그 'tr'을 행에
            rows = div_history.find_all('tr')
            for row in rows[1:]:
                cell = row.find_all('td')
                data['일자'].append(cell[0].get_text(strip=True))
                data['종가'].append(cell[1].get_text(strip=True))
                data['전일비'].append(cell[2].get_text(strip=True))
                data['등락률'].append(cell[3].get_text(strip=True))
                data['거래량'].append(cell[4].get_text(strip=True))
                data['거래대금'].append(cell[5].get_text(strip=True))
                data['개인(억)'].append(cell[6].get_text(strip=True))
                data['외국인(억)'].append(cell[7].get_text(strip=True))
                data['기관(억)'].append(cell[8].get_text(strip=True))
            


            # 데이터프레임 생성 후 결과 데이터프레임에 추가
            df = pd.DataFrame(data)
            result_df_11to530 = pd.concat([result_df_11to530, df], ignore_index=True)

    # 페이지가 12의 배수일 때 다음 페이지로 넘어가기
        if i % 12 == 0:
            next_button = driver.find_element(By.CSS_SELECTOR, '#boxDailyHistory > div.box_contents > div > div > a.btnNext')
            next_button.click()
            time.sleep(1)

# 결과 데이터프레임 출력
result_df_11to530.head()

,일자,종가,전일비,등락률,거래량,거래대금,개인(억),외국인(억),기관(억)
0,23.10.26,"2,299.08",▼64.09,-2.71%,"531,073","9,251,519","3,227","-4,789","1,108"
1,23.10.25,"2,363.17",▼20.34,-0.85%,"440,807","7,832,203","3,618","-3,426",-631
2,23.10.24,"2,383.51",▲26.49,+1.12%,"444,165","7,949,285",715,"-1,552",362
3,23.10.23,"2,357.02",▼17.98,-0.76%,"395,930","7,117,567","1,646","-2,563",705
4,23.10.20,"2,375.00",▼40.80,-1.69%,"521,208","9,295,958","1,118","2,119","-1,758"


In [10]:
len(result_df_11to530)

5200

result_df_1to10과 result_df_11to530 데이터프레임 합치기

In [12]:
total_result_df = pd.concat([result_df_1to10, result_df_11to530])

total_result_df

,일자,종가,전일비,등락률,거래량,거래대금,개인(억),외국인(억),기관(억)
0,24.03.22,"2,748.56",▼6.30,-0.23%,"501,319","11,385,951","-5,299","6,649","-1,630"
1,24.03.21,"2,754.86",▲64.72,+2.41%,"825,227","14,013,557","-29,144","18,706","10,506"
2,24.03.20,"2,690.14",▲33.97,+1.28%,"720,452","12,739,395","-14,054","8,516","5,866"
3,24.03.19,"2,656.17",▼29.67,-1.10%,"695,866","10,702,805","9,226","-2,388","-7,013"
4,24.03.18,"2,685.84",▲19.00,+0.71%,"408,182","9,523,806","-1,865",-847,"2,254"
...,...,...,...,...,...,...,...,...,...
5195,02.10.22,639.27,▼13.40,-2.05%,"1,094,948","2,671,657",0,0,0
5196,02.10.21,652.67,▼18.12,-2.70%,"1,622,877","2,713,863",0,0,0
5197,02.10.18,670.79,▲26.13,+4.05%,"1,160,610","3,701,835",0,0,0
5198,02.10.17,644.66,▲8.41,+1.32%,"1,290,310","3,050,903",0,0,0


In [13]:
len(total_result_df)

5300

In [15]:
#필요없는 열 삭제
del total_result_df['전일비']
del total_result_df['등락률']
del total_result_df['거래량']

KeyError: '전일비'

In [16]:
import re

# '일자' 열에서 '02.'으로 시작하는 행만 필터링
remove_row_12 = total_result_df[total_result_df['일자'].str.contains(r'^24\.', regex=True)]

# 결과 확인
remove_row_12.head()

,일자,종가,거래대금,개인(억),외국인(억),기관(억)
0,24.03.22,"2,748.56","11,385,951","-5,299","6,649","-1,630"
1,24.03.21,"2,754.86","14,013,557","-29,144","18,706","10,506"
2,24.03.20,"2,690.14","12,739,395","-14,054","8,516","5,866"
3,24.03.19,"2,656.17","10,702,805","9,226","-2,388","-7,013"
4,24.03.18,"2,685.84","9,523,806","-1,865",-847,"2,254"


In [17]:
import re

# '일자' 열에서 '02.'으로 시작하는 행만 필터링
remove_row_02 = total_result_df[total_result_df['일자'].str.contains(r'^02\.', regex=True)]

# 결과 확인
remove_row_02.head()

,일자,종가,거래대금,개인(억),외국인(억),기관(억)
5148,02.12.30,627.55,"3,431,175",0,0,0
5149,02.12.27,656.92,"1,368,616",0,0,0
5150,02.12.26,671.89,"1,570,391",0,0,0
5151,02.12.24,677.82,"1,664,618",0,0,0
5152,02.12.23,691.38,"2,064,844",0,0,0


In [18]:
# '일자' 열의 값이 '02.'으로 시작하는 행을 제거하여 새로운 데이터프레임 생성
total_result_df = total_result_df[~total_result_df['일자'].str.contains(r'^02\.', regex=True)]
total_result_df = total_result_df[~total_result_df['일자'].str.contains(r'^24\.', regex=True)]

# 결과 확인
total_result_df

,일자,종가,거래대금,개인(억),외국인(억),기관(억)
56,23.12.28,"2,655.28","9,418,930","-13,571","6,948","6,603"
57,23.12.27,"2,613.50","10,359,764","2,043","2,261","-4,401"
58,23.12.26,"2,602.59","9,582,766","-10,522",782,"9,702"
59,23.12.22,"2,599.51","8,848,288","-2,059",-367,"2,386"
60,23.12.21,"2,600.02","9,467,809",-545,120,397
...,...,...,...,...,...,...
5143,03.01.08,651.72,"1,488,455",0,0,0
5144,03.01.07,652.20,"1,869,970",0,0,0
5145,03.01.06,666.71,"1,598,625",0,0,0
5146,03.01.03,661.10,"2,270,513",0,0,0


In [19]:
total_result_df

,일자,종가,거래대금,개인(억),외국인(억),기관(억)
56,23.12.28,"2,655.28","9,418,930","-13,571","6,948","6,603"
57,23.12.27,"2,613.50","10,359,764","2,043","2,261","-4,401"
58,23.12.26,"2,602.59","9,582,766","-10,522",782,"9,702"
59,23.12.22,"2,599.51","8,848,288","-2,059",-367,"2,386"
60,23.12.21,"2,600.02","9,467,809",-545,120,397
...,...,...,...,...,...,...
5143,03.01.08,651.72,"1,488,455",0,0,0
5144,03.01.07,652.20,"1,869,970",0,0,0
5145,03.01.06,666.71,"1,598,625",0,0,0
5146,03.01.03,661.10,"2,270,513",0,0,0


In [27]:
# 일자 / 종가 / 거래대금

total_result_df_01 = total_result_df[['일자', '종가', '거래대금']].copy()

total_result_df_01

,일자,종가,거래대금
56,23.12.28,"2,655.28","9,418,930"
57,23.12.27,"2,613.50","10,359,764"
58,23.12.26,"2,602.59","9,582,766"
59,23.12.22,"2,599.51","8,848,288"
60,23.12.21,"2,600.02","9,467,809"
...,...,...,...
5143,03.01.08,651.72,"1,488,455"
5144,03.01.07,652.20,"1,869,970"
5145,03.01.06,666.71,"1,598,625"
5146,03.01.03,661.10,"2,270,513"


In [28]:
total_result_df_01['거래대금'] = total_result_df_01['거래대금'].str.replace(',', '').copy()

total_result_df_01

,일자,종가,거래대금
56,23.12.28,"2,655.28",9418930
57,23.12.27,"2,613.50",10359764
58,23.12.26,"2,602.59",9582766
59,23.12.22,"2,599.51",8848288
60,23.12.21,"2,600.02",9467809
...,...,...,...
5143,03.01.08,651.72,1488455
5144,03.01.07,652.20,1869970
5145,03.01.06,666.71,1598625
5146,03.01.03,661.10,2270513


In [29]:
#뒤 숫자 3개 제거
def remove_last_three_num(number_str):
    # 마지막 3자리 제거
    number_str_without_commas = number_str.replace(',', '')
    return int(number_str_without_commas[:-3])

total_result_df_01['거래대금'] = total_result_df_01['거래대금'].apply(remove_last_three_num).copy()

total_result_df_01

,일자,종가,거래대금
56,23.12.28,"2,655.28",9418
57,23.12.27,"2,613.50",10359
58,23.12.26,"2,602.59",9582
59,23.12.22,"2,599.51",8848
60,23.12.21,"2,600.02",9467
...,...,...,...
5143,03.01.08,651.72,1488
5144,03.01.07,652.20,1869
5145,03.01.06,666.71,1598
5146,03.01.03,661.10,2270


In [22]:
# 일자 / 종가 / 개인-외국인-기관

total_result_df_02 = total_result_df[['일자', '종가', '개인(억)', '외국인(억)', '기관(억)']].copy()

total_result_df_02

,일자,종가,개인(억),외국인(억),기관(억)
56,23.12.28,"2,655.28","-13,571","6,948","6,603"
57,23.12.27,"2,613.50","2,043","2,261","-4,401"
58,23.12.26,"2,602.59","-10,522",782,"9,702"
59,23.12.22,"2,599.51","-2,059",-367,"2,386"
60,23.12.21,"2,600.02",-545,120,397
...,...,...,...,...,...
5143,03.01.08,651.72,0,0,0
5144,03.01.07,652.20,0,0,0
5145,03.01.06,666.71,0,0,0
5146,03.01.03,661.10,0,0,0


In [33]:
import mysql.connector

remote = mysql.connector.connect(
    host = 'database-1.cduo2y2a8r8j.ap-northeast-2.rds.amazonaws.com',
    port = 3306,
    user = 'root',
    password = 'clover12',
    database = 'amrbase'
)

In [34]:
sql = """INSERT INTO KOSPI_MARKET_01 VALUES (%s, %s, %s)"""
cursor = remote.cursor(buffered = True)

In [35]:
for i, row in total_result_df_01.iterrows():
    cursor.execute(sql, tuple(row))

    print(tuple(row))
    remote.commit()

('23.12.28', '2,655.28', 9418)
('23.12.27', '2,613.50', 10359)
('23.12.26', '2,602.59', 9582)
('23.12.22', '2,599.51', 8848)
('23.12.21', '2,600.02', 9467)
('23.12.20', '2,614.30', 11202)
('23.12.19', '2,568.55', 8418)
('23.12.18', '2,566.86', 10181)
('23.12.15', '2,563.56', 12873)
('23.12.14', '2,544.18', 12960)
('23.12.13', '2,510.66', 7513)
('23.12.12', '2,535.27', 7732)
('23.12.11', '2,525.36', 8260)
('23.12.08', '2,517.85', 7916)
('23.12.07', '2,492.07', 8127)
('23.12.06', '2,495.38', 7685)
('23.12.05', '2,494.28', 8300)
('23.12.04', '2,514.95', 8772)
('23.12.01', '2,505.01', 8837)
('23.11.30', '2,535.29', 11992)
('23.11.29', '2,519.81', 9428)
('23.11.28', '2,521.76', 8753)
('23.11.27', '2,495.66', 8376)
('23.11.24', '2,496.63', 6537)
('23.11.23', '2,514.96', 6577)
('23.11.22', '2,511.70', 7755)
('23.11.21', '2,510.42', 7713)
('23.11.20', '2,491.20', 6586)
('23.11.17', '2,469.85', 8129)
('23.11.16', '2,488.18', 6806)
('23.11.15', '2,486.67', 9328)
('23.11.14', '2,433.25', 6382)
('

In [38]:
sql_2 = """INSERT INTO KOSPI_MARKET_02 VALUES (%s, %s, %s, %s, %s)"""
cursor = remote.cursor(buffered=True)

In [ ]:
for i, row in total_result_df_02.iterrows():
    cursor.execute(sql, tuple(row))

    print(tuple(row))
    remote.commit()